In [3]:
# from jupyter_dash import JupyterDash
# from dash import html, dcc, Input, Output
# import plotly.graph_objs as go
# import pandas as pd

import pandas as pd
import datetime as dt
import os
import statistics
import plotly.graph_objects as go
import plotly_express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [12]:
# formatting old
main_text_colour = '#62466B' #darker purple
secondary_text_colour = '#A7ACD9' #lighter purple
marker_colour = '#93C6D6' #blue
comp_colour_1 = '#82AEB1' #lighter green
com_colour_2 = '#668586' #darker green

# new colors
dash_red = '#FA897B'
dash_yellow = '#FFDD94'
dash_blue = '#86E3CE'
dash_green = '#D0E6A5'
dash_purple = '#CCABDB'

dash_red_light = '#FBBAB2'
dash_yellow_light = '#F7EACE'
dash_blue_light = '#BDFFF0'
dash_green_light = '#E2EEC9'
dash_purple_light ='#D7CADD'


In [120]:
GOAL_WEIGHT = 62


In [100]:
meas_data = pd.read_csv('../data/myfitnesspal/cleaned/mfp_measurements_cleaned.csv')
print(meas_data)
print(meas_data['Weight'].iloc[1], meas_data['Weight'].iloc[-1])

           Date     Weight  Daily_change  weekly_avg_weight  weekly_loss  \
0    2024-02-26  96.600000           NaN                NaN          NaN   
1    2024-02-27  96.600000      0.000000                NaN          NaN   
2    2024-02-28  96.233333     -0.366667                NaN          NaN   
3    2024-02-29  95.866667     -0.366667                NaN          NaN   
4    2024-03-01  95.500000     -0.366667                NaN          NaN   
..          ...        ...           ...                ...          ...   
486  2025-06-26  68.300000      0.500000                NaN          NaN   
487  2025-06-27  67.100000     -1.200000                NaN          NaN   
488  2025-06-28  67.566667      0.466667                NaN          NaN   
489  2025-06-29  68.033333      0.466667          68.014286    -0.565714   
490  2025-06-30  68.500000      0.466667                NaN          NaN   

     Weight lbs  Daily_change lbs  weekly_avg_weight lbs  weekly_loss lbs  
0    212.96

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

my_dash_app_style = {
    "color": "gray",
    "background-color": "#f8f8ff",
    "font-family": "Arial",
    "text-align": "left",
}

header_container = dbc.Row(
    [
        dbc.Col(html.H1(f"Total Loss {meas_data['Weight'].iloc[0] - meas_data['Weight'].iloc[-1]}", id="total-loss-title", style={"color": 'darkgrey'})),
        dbc.Col(
            dbc.Card(
                [ html.Div("Units", style={"fontWeight": "bold", "marginBottom": "5px", "color": main_text_colour, "fontSize": "16px"}),
                dcc.RadioItems(
                id='unit-toggle',
                options=[{'label': 'kg', 'value': 'kg'}, {'label': 'lb', 'value': 'lb'}],
                value='kg',
                labelStyle={'display': 'inline-block', 'margin-right': '10px', 'color': main_text_colour, 'font-size': '20px'},
                inputStyle={'margin-right': '5px'}
            )
            ],
            body=True,
            style={"background-color": "white", "padding": "10px", "width": "150px"}
            ),
            width="auto",
            style={"margin-left": "auto"}
        )
    ],
        style={"margin-bottom": "20px"}
        )

kpi_row = dbc.Row(
    [
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H6("Current Weight", className="card-title", style={"color": "grey", "marginBottom": "5px"}),
                    html.H4("75.5 kg", className="card-value", style={"fontWeight": "bold"})
                ]),
                style={
                    "backgroundColor": "#f8f8ff",
                    "height": "150px",
                    "borderRadius": "10px",
                    "boxShadow": "0px 4px 8px rgba(0,0,0,0.05)"
                }
            ),
            width=6  # Half of the row
        ),
        dbc.Col(
            dbc.Card(
                dbc.CardBody([
                    html.H6("Weight Lost", className="card-title", style={"color": "grey", "marginBottom": "5px"}),
                    html.H4("30.1 kg", className="card-value", style={"fontWeight": "bold"})
                ]),
                style={
                    "backgroundColor": "#f8f8ff",
                    "height": "150px",
                    "borderRadius": "10px",
                    "boxShadow": "0px 4px 8px rgba(0,0,0,0.05)"
                }
            ),
            width=6
        )
    ],
    style={"marginBottom": "10px"}
)

static_analytics_container = dbc.Row([
                                # Left hand side is main chart visualisation of weight over time
                                dbc.Col(
                                    dbc.Card(
                                        dbc.CardBody([
                                            html.H4("Weight Over Time", className="card-title", style={"color": 'darkgrey'}),
                                            dcc.Graph(id='weight-graph')
                                        ]),
                                        style={
                                            "backgroundColor": "white",
                                            "padding": "10px",
                                            "margin": "10px",
                                            "boxShadow": "0px 4px 12px rgba(0,0,0,0.1)",
                                            "borderRadius": "5px",
                                            "marginBottom": "5px",  # reduce bottom margin (default is usually larger)
                                            "paddingBottom": "0"
                                        }
                                    ), width = 8
                                ),                                     
                                dbc.Col(
                                    dbc.Card(
                                        dbc.CardBody([
                                            html.H5("Key Stats", className="card-title", style={"color": 'darkgrey'}),

                                            # ⬇️ Horizontal KPI cards inside "Key Stats"
                                              # space below KPI row

                                            # ⬇️ Optional small chart
                                            dcc.Graph(id='percentage-chart'),
                                            dbc.Row([
                                                dbc.Col(
                                                    dbc.Card(
                                                        dbc.CardBody([
                                                            html.P("Starting Weight", className="card-subtitle"),
                                                            html.P(id='kpi-start-weight', className="card-text", style={"fontSize": "20px", "fontWeight": "bold"})
                                                        ]),
                                                        style={"backgroundColor": "#f1f1f1", "textAlign": "center", "borderRadius": "5px"}
                                                    )
                                                ),
                                                dbc.Col(
                                                    dbc.Card(
                                                        dbc.CardBody([
                                                            html.P("Current Weight", className="card-subtitle"),
                                                            html.P(id='kpi-current-weight', className="card-text", style={"fontSize": "20px", "fontWeight": "bold"})
                                                        ]),
                                                        style={"backgroundColor": "#f1f1f1", "textAlign": "center", "borderRadius": "5px"}
                                                    )
                                                )
                                            ], className="mb-3"),
                                        ]),
                                        style={
                                            "backgroundColor": "#ffffff",
                                            "padding": "10px",
                                            "margin": "10px",
                                            "boxShadow": "0px 4px 12px rgba(0,0,0,0.05)",
                                            "borderRadius": "5px"
                                        }
                                    ),
                                    width=4
                                )                     
                                
])

app.layout = html.Div([
    header_container,
    static_analytics_container
], style={"backgroundColor": "#f8f8ff", "padding": "10px"}
                      )



# Callback function for main chart
@app.callback(
    Output('weight-graph', 'figure'),
    Output('total-loss-title','children'),
    Input('unit-toggle', 'value')
)

def update_graphs_and_kpis(unit):
    fig = go.Figure()
    if unit == 'kg': 
        fig.add_trace(go.Scatter(x=meas_data['Date'], y=meas_data['Weight'], mode='lines', name='Weight (kg)', 
                                 line=dict(shape='spline',color=dash_blue, width=1),fill='tozeroy',fillgradient=dict(
                                    type="vertical",
                                    colorscale=[(0.0,'white'), (0.5,'white'), (1.0, dash_blue)],
            ),))   
        fig.add_trace(go.Scatter(x=meas_data['Date'], y=meas_data['weekly_avg_weight'], mode='markers',name='Weekly Average Weight (kg)', 
                                 marker=dict(color=dash_blue, size=4)))
        fig.update_layout(yaxis=dict(title=f'Weight ({unit})',title_standoff=8,ticklen=15,automargin=True),
                      xaxis=dict(title_standoff=8,ticklen=15,range=[meas_data['Date'].min(),meas_data['Date'].max()],automargin=True),
                      plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',
                      legend=dict(
                            x=0.75,               # X position (1.0 is the far right of the plot area)
                            y=1,                  # Y position (1.0 is the top)
                            xanchor='left',       # Anchors the legend box to the left of x=1.02
                            yanchor='top',        # Anchors the legend box to the top of y=1
                            bgcolor='rgba(255,255,255,0.7)',  # Optional: semi-transparent background
                            bordercolor='rgba(0,0,0,0)',
                            borderwidth=1),
                            margin=dict(l=40, r=20, t=40, b=40),
                            yaxis_range=[meas_data['Weight'].min(),meas_data['Weight'].max()]
                         
                            
    )
        total_loss = meas_data['Weight'].iloc[0] - meas_data['Weight'].iloc[-1]
    else:
        
        fig.add_trace(go.Scatter(x=meas_data['Date'], y=meas_data['Weight lbs'], mode='lines', name='Weight (lbs)', 
                                 line=dict(shape='spline',color=dash_blue, width=1),fill='tozeroy',fillgradient=dict(
                                    type="vertical",
                                    colorscale=[(0.0,'white'), (0.5,'white'), (1.0, dash_blue)],
            ),))   
        fig.add_trace(go.Scatter(x=meas_data['Date'], y=meas_data['weekly_avg_weight lbs'], mode='markers',name='Weekly Average Weight (lbs)', 
                                 marker=dict(color=dash_blue, size=4)))
        fig.update_layout(yaxis=dict(title=f'Weight ({unit})',title_standoff=8,ticklen=15,automargin=True),
                      xaxis=dict(title_standoff=8,ticklen=15,range=[meas_data['Date'].min(),meas_data['Date'].max()],automargin=True),
                      plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',
                      legend=dict(
                            x=0.75,               # X position (1.0 is the far right of the plot area)
                            y=1,                  # Y position (1.0 is the top)
                            xanchor='left',       # Anchors the legend box to the left of x=1.02
                            yanchor='top',        # Anchors the legend box to the top of y=1
                            bgcolor='rgba(255,255,255,0.7)',  # Optional: semi-transparent background
                            bordercolor='rgba(0,0,0,0)',
                            borderwidth=1),
                            margin=dict(l=40, r=20, t=40, b=40),
                            yaxis_range=[meas_data['Weight lbs'].min(),meas_data['Weight lbs'].max()]
                            
    )
        total_loss = meas_data['Weight lbs'].iloc[0] - meas_data['Weight lbs'].iloc[-1]
        
    
    return fig,total_loss

# Callback function for KPI cards and bar chart
@app.callback(
    Output('percentage-chart', 'figure'),
    Output('kpi-start-weight', 'children'),
    Output('kpi-current-weight', 'children'),
    Input('unit-toggle', 'value')
)
def update_percentage_chart(unit):
    # Example placeholder logic
    if unit == 'kg':
        unit_label='kg'
        y_data = meas_data['Weight']
        percent_lost = ((meas_data['Weight'].iloc[0] - meas_data['Weight'].iloc[-1]) / (meas_data['Weight'].iloc[0] - GOAL_WEIGHT)) * 100
        start = y_data.iloc[0]
        current = y_data.iloc[-1]
        goal = GOAL_WEIGHT
    else:
        unit_label='lb'
        y_data = meas_data['Weight lbs']
        percent_lost = ((meas_data['Weight'].iloc[0] - meas_data['Weight'].iloc[-1]) / (meas_data['Weight'].iloc[0] - GOAL_WEIGHT)) * 100
        start = y_data.iloc[0]
        current = y_data.iloc[-1]
        goal = GOAL_WEIGHT * 2.2046226218
        
        
    start_weight = f'{y_data.iloc[0]:.1f} {unit_label}'
    current_weight = f'{y_data.iloc[-1]:.1f} {unit_label}'
    
    
    fig = go.Figure(go.Bar(
        x=[percent_lost],
        y=[""],
        orientation='h',
        marker=dict(color=dash_purple),
        text=f"{percent_lost:.1f}%",
        textfont=dict(color='white', size=16),
        textposition="inside", 
        insidetextanchor='end', 
        hoverinfo='x'
        ))
    fig.add_annotation(
        x=percent_lost,
        y=-1,  # position below the axis (adjust as needed)
        xref='x',
        yref='paper',
        text=f"<b>{current_weight}</b>",
        showarrow=False,
        font=dict(size=16, color=dash_purple),
        align='center'
    )
    fig.add_annotation(
        x=percent_lost,
        y=-1.4,  # position below the axis (adjust as needed)
        xref='x',
        yref='paper',
        text="(Current)",
        showarrow=False,
        font=dict(size=12, color='black'),
        align='center'
    )
    fig.update_layout(
        shapes=[
            # vertical line at start (0)
            dict(type="line", x0=0, x1=0, y0=-0.1, y1=0.1, xref='x', yref='paper',
                line=dict(color="gray", width=1)),
            # vertical line at current (percent_lost)
            dict(type="line", x0=percent_lost, x1=percent_lost, y0=-0.1, y1=0.1, xref='x', yref='paper',
                line=dict(color="gray", width=1)),
            # vertical line at goal (100)
            dict(type="line", x0=100, x1=100, y0=-0.1, y1=0.1, xref='x', yref='paper',
                line=dict(color="gray", width=1)),
        ]
    )

    fig.update_layout(
            title=f"You're {percent_lost:.1f}% of the way to your goal!",
            title_font_size=12,
            xaxis=dict(range=[0, 100], showgrid=False, title="",tickvals=[0, percent_lost, 100],        
            ticktext=[start_weight, "", f"{goal:.1f}{unit_label}"],
            showticklabels=True),
            yaxis=dict(showticklabels=False),
            height=100,
            margin=dict(l=20, r=20, t=30, b=40),
            plot_bgcolor='lightgrey',
            paper_bgcolor='rgba(0,0,0,0)'
        )
        
        
    return fig, start_weight, current_weight


if __name__ == "__main__":
    app.run_server(debug=True,port=8051)